# 기출 7회_작업형1

## 문제1
결측치가 있는 행을 제거한 후, 학생이 가장 많이 수강한 과목을 찾고, 해당 과목 점수를 표준화(스탠다스 스케일) 한 뒤에 표준화된 가장 큰 값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [72]:
# My)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_1/student_assessment.csv")

df = df.dropna()
# print(df.info())

grouped = df.groupby(['id_assessment'])['student_id'].count()
top = grouped.idxmax()

# print(grouped)
cond = df['id_assessment'] == top
df1 = df[cond].copy()
# print(df1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df1['score'] = scaler.fit_transform(df1[['score']])
# print(df1)
print(round(df1.nlargest(1, 'score').iloc[-1]['score'], 3))

2.183


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_1/student_assessment.csv")

# 0) 데이터 확인
print("결측치 수:", df.isnull().sum().sum())

# 1) 결측치 제거
print(df.shape)
df = df.dropna()
print(df.shape)

# 2) 가장 많이 수강한 과목 필터링
id = df['id_assessment'].value_counts().idxmax()
cond = df['id_assessment'] == id
df = df[cond]

# 3) 과목 점수 스탠다드 스케일
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['score'] = scaler.fit_transform(df[['score']])

# 4) 가장 큰 값
print(round(df['score'].max(), 3))

결측치 수: 21
(2565, 4)
(2544, 4)
2.183


## 문제2
DE1~DE77컬럼 중 주가지수의 종가 "close"와 가장 상관관계가 높은 변수를 찾아, 해당 변수의 평균값을 구하시오. (반올림하여 소수 넷째자리까지 계산)

In [85]:
# My)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_1/stock_market.csv")

df_corr = df.corr()['close'].abs()
col = df_corr.loc['DE1':'DE77'].idxmax()
# print(col)
print(round(df[col].mean(), 4))

-0.0004


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_1/stock_market.csv")

# 1) close와의 상관관계(절대값)
df_corr = df.corr()['close'].abs()

# 2) 상관관계가 높은 변수명
col = df_corr.loc['DE1':'DE77'].idxmax()

# 3) '2'에서 구한 변수명의 평균값
print(round(df[col].mean(), 4))

-0.0004


## 문제3
IQR을 이용해 이산화탄소(CO2) 이상치 수를 찾으시오

- Q1 (제 1사분위수): 하위 25% 값
- Q3 (제 3사분위수): 상위 25% 값
- IQR (사분위수 범위): IQR = Q3 - Q1

- lower (하한선): Q1 - 1.5 * IQR, 이 값보다 작은 데이터는 이상치로 간주
- upper (상한선): Q3 + 1.5 * IQR, 이 값보다 큰 데이터는 이상치로 간주



In [92]:
# My) 

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_1/air_quality.csv")

# print(df)
Q1 = df['CO2'].quantile(.25)
Q3 = df['CO2'].quantile(.75)
IQR = Q3 - Q1

lower = Q1-1.5*IQR
upper = Q3+1.5*IQR

cond1 = df['CO2'] < lower
cond2 = df['CO2'] > upper
outliers = df[cond1 | cond2]

# print(Q1, Q3, IQR)
print(len(outliers))

304


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_1/air_quality.csv")

# 1) IQR 계산
Q1 = df['CO2'].quantile(0.25)
Q3 = df['CO2'].quantile(0.75)
IQR = Q3 - Q1

# 2) 상한 및 하한 계산
upper = Q3 + 1.5 * IQR
lower = Q1 - 1.5 * IQR

# 3) 이상치 조건
cond1 = df['CO2'] < lower
cond2 = df['CO2'] > upper
outliers = df[cond1 | cond2]

# 4) 이상치 수 계산
print(len(outliers))

304


# 기출 7회_작업형2
## mart 판매 데이터를 기반으로 판매액을 예측
- 제공된 데이터 목록: mart_train.csv (훈련 데이터), mart_test.csv (평가용 데이터)
- 예측할 컬럼: total (총 판매액)
학습용 데이터(mart_train.csv)를 이용하여 총 판매액을 예측하는 모델을 만든 후, 이를 평가용 데이터(mart_test.csv)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV 파일로 생성하시오.
- 제출 파일은 다음 한 개의 컬럼을 포함해야 합니다.
- pred: 예측된 총 판매액
- 제출 파일명: 'result.csv' 제출한 모델의 성능은 RMSE(Root Mean Square Error) 평가지표에 따라 채점한다.
- 제출 CSV 파일명 및 형태: result.csv

~~~
pred
10000
20000
30000
40000
...
~~~

## 답안 제출 참고
- pd.read_csv('result.csv') 로 제출 코드 확인

## 📌 My Code

In [20]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_2/mart_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_2/mart_test.csv")

# 1. 데이터 유형 파악
# 결측치: X
# 범주형: ['branch', 'city', 'customer_type', 'gender', 'product_line', 'payment_method', 'time_of_day', 'day_name']
# target: total
# print(train.info())
# print(test.info())
# print(train)
# print(train.shape, test.shape) # (700, 10) (300, 9)
# print(train.select_dtypes('object').columns)

# 2. 데이터 전처리 (4)
# (1) X_full
X = train.drop(['total'], axis=1)
y = train['total']
X_full = pd.concat([X, test], axis=0)

# (2) 결측치: X

# (3) 수치형 변수 스케일링: X

# (4) 범주형 변수 인코딩
X_full = pd.get_dummies(X_full)
# print(X_full)
# print(X_full.shape) # (1000, 30)

# 3. 데이터 분리 (X_train, X_test / train_test_split)
X_train = X_full[:train.shape[0]]
X_test = X_full[train.shape[0]:]
# print(X_train.shape, X_test.shape) # (700, 30) (300, 30)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape) # (560, 30) (140, 30) (560,) (140,)

# 4. 모델 학습
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor()
RF.fit(X_train, y_train)
y_val_pred_rf = RF.predict(X_val)
# print(y_val_pred_rf)
# print(y_val_pred_rf.shape) # (140,)

from sklearn.ensemble import HistGradientBoostingRegressor
HGB = HistGradientBoostingRegressor()
HGB.fit(X_train, y_train)
y_val_pred_hgb = HGB.predict(X_val)
# print(y_val_pred_hgb)
# print(y_val_pred_hgb.shape) # (140,)

# 5. 모델 평가
from sklearn.metrics import root_mean_squared_error
score_rf = root_mean_squared_error(y_val, y_val_pred_rf)
print('score_rf', score_rf)

score_hgb = root_mean_squared_error(y_val, y_val_pred_hgb)
print('score_hgb', score_hgb)

# 6. 결과 저장
y_pred = RF.predict(X_test)
# print(y_pred)
# print(y_pred.shape) # (300,)

result = pd.DataFrame({'pred': y_pred})
result.to_csv('result_72.csv', index=False)

check = pd.read_csv('result_72.csv')
print(check)
# print(check.shape) # (300, 1)

score_rf 379654.48854407197
score_hgb 409161.0203967835
            pred
0    356511.0150
1    674935.3800
2    263410.7175
3    593099.0100
4    438679.7100
..           ...
295  519270.0975
296  435027.1275
297  475343.0325
298  396731.0025
299  494153.1000

[300 rows x 1 columns]


# 작업형2 풀이

## 1. 문제정의
- RMSE
- target: total
- 예측 파일명: result.csv
- 컬럼 1개(pred)

## 2. 라이브러리 및 데이터 불러오기

In [21]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_2/mart_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_2/mart_test.csv")

## 3. 탐색적 데이터 분석(EDA)

In [22]:
# 데이터 크기 확인
train.shape, test.shape

((700, 10), (300, 9))

In [23]:
# 샘플 확인
train.head(3)

branch       city customer_type  gender            product_line      total  \
0      A     Yangon        Member  Female       Health and beauty  823457.25   
1      C  Naypyitaw        Normal  Female  Electronic accessories  120330.00   
2      A     Yangon        Normal    Male      Home and lifestyle  510788.25   

  payment_method  rating time_of_day  day_name  
0        Ewallet     9.1   afternoon  Saturday  
1           Cash     9.6     morning    Friday  
2    Credit card     7.4   afternoon    Sunday

In [24]:
test.head(3)

branch       city customer_type  gender         product_line payment_method  \
0      C  Naypyitaw        Normal  Female  Fashion accessories        Ewallet   
1      B   Mandalay        Normal    Male   Food and beverages    Credit card   
2      B   Mandalay        Member  Female  Fashion accessories    Credit card   

   rating time_of_day   day_name  
0     9.6   afternoon   Thursday  
1     4.3     evening  Wednesday  
2     5.0     evening  Wednesday

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   branch          700 non-null    object 
 1   city            700 non-null    object 
 2   customer_type   700 non-null    object 
 3   gender          700 non-null    object 
 4   product_line    700 non-null    object 
 5   total           700 non-null    float64
 6   payment_method  700 non-null    object 
 7   rating          700 non-null    float64
 8   time_of_day     700 non-null    object 
 9   day_name        700 non-null    object 
dtypes: float64(2), object(8)
memory usage: 54.8+ KB


In [26]:
# 기초 통계값 확인
train.describe()

total      rating
count  7.000000e+02  700.000000
mean   4.850780e+05    7.003429
std    3.643907e+05    1.713078
min    1.904175e+04    4.000000
25%    2.001195e+05    5.500000
50%    3.818745e+05    7.000000
75%    7.061276e+05    8.425000
max    1.563975e+06   10.000000

In [27]:
train.describe(include='O')

branch    city customer_type gender       product_line payment_method  \
count     700     700           700    700                700            700   
unique      3       3             2      2                  6              3   
top         A  Yangon        Normal   Male  Sports and travel           Cash   
freq      236     236           354    356                127            246   

       time_of_day  day_name  
count          700       700  
unique           3         7  
top        evening  Saturday  
freq           309       114

In [28]:
test.describe(include='O')

branch    city customer_type  gender            product_line  \
count     300     300           300     300                     300   
unique      3       3             2       2                       6   
top         A  Yangon        Member  Female  Electronic accessories   
freq      104     104           155     157                      58   

       payment_method time_of_day  day_name  
count             300         300       300  
unique              3           3         7  
top           Ewallet     evening  Saturday  
freq              110         123        50

In [29]:
train.isnull().sum().sum()

np.int64(0)

In [30]:
test.isnull().sum().sum()

np.int64(0)

In [31]:
train['total'].value_counts()

total
283641.75     2
263875.50     2
415422.00     2
326450.25     2
130851.00     2
             ..
293391.00     1
137103.75     1
348232.50     1
104107.50     1
1535625.00    1
Name: count, Length: 695, dtype: int64

In [32]:
pd.set_option('display.float_format', '{:.10f}'.format)
train['total'].describe()

count       700.0000000000
mean     485078.0175000000
std      364390.7265411940
min       19041.7500000000
25%      200119.5000000000
50%      381874.5000000000
75%      706127.6250000000
max     1563975.0000000002
Name: total, dtype: float64

## 4. 데이터 전처리

In [33]:
target = train.pop('total')

In [34]:
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(700, 9) (300, 9)
(700, 30) (300, 30)


## 5. 검증 데이터 분할

In [35]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((560, 30), (140, 30), (560,), (140,))

## 6. 머신러닝 학습 및 평가

In [36]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**0.5

In [37]:
# 선형회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
rmse(y_val, pred)

np.float64(374016.3661648522)

In [38]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
rmse(y_val, pred)

np.float64(385951.8053430962)

In [ ]:
# Xgboost
import xgboost as xgb
xg = xgb.XGBRegressor(random_state=0)
xg.fit(X_tr, y_tr)
pred = xg.predict(X_val)
rmse(y_val, pred)

In [ ]:
# LightGBM
import lightgbm as lgb
lg = lgb.LGBMRegressor(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
rmse(y_val, pred)

## 7. 예측 및 결과 파일 생성

In [44]:
pred = lr.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv("result.csv", index=False)

In [45]:
pd.read_csv('result.csv')

pred
0   485484.5740277334
1   474683.4165377701
2   486669.1574532852
3   426407.8222290425
4   599942.9042138030
..                ...
295 501132.5887398274
296 554273.9221195823
297 540502.7549631427
298 550704.8821613132
299 506543.6684025059

[300 rows x 1 columns]

## 8. 성능 개선

In [46]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_2/mart_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_2/mart_test.csv")

# target 데이터
target = train.pop('total')

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 선형회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
print(rmse(y_val, pred))

# 최종 제출 파일
pred = lr.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv("result.csv", index=False)

# 베이스라인(원핫 인코딩): 374016.3661648522
# 레이블 인코딩: 371070.1344734549

371070.1344734549


# 기출 7회_작업형3

## 문제1
주어진 조개 데이터 300개 중 앞에서부터 210개는 train 데이터로 만들고, 나머지 90개는 test데이터로 만든다. 모델을 학습(적합)할 때는 train데이터를 사용하고, 예측할 때는 test데이터를 사용한다. 모델은 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0:암컷, 1:수컷)

### 문제1-1. weight를 독립변수로 gender를 종속변수로 사용하여 로지스틱 회귀 모형을 만들고, weight 변수가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/clam.csv")

# 데이터셋 분할
train = df.iloc[:210]
test = df.iloc[210:]
print(train.shape, test.shape)

(210, 6) (90, 6)


In [ ]:
from statsmodels.formula.api import logit
import numpy as np

# 1) 로지스틱 회귀 모델 생성 및 학습
model = logit("gender ~ weight", data=train).fit()
print(model.summary())

# 2) 오즈비 계산
print(round(np.exp(0.0047),4))
print(round(np.exp(model.params['weight']),4))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      208
Method:                           MLE   Df Model:                            1
Date:                Sat, 21 Sep 2024   Pseudo R-squ.:                0.003431
Time:                        05:54:09   Log-Likelihood:                -144.91
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.3178
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3140      0.276     -1.137      0.256      -0.855       0.227
weight         0.0047      0.

In [ ]:
# [참고] <심화학습> 암컷의 오즈비
female_odds_ratio = round(np.exp(-model.params['weight']), 4)
print(female_odds_ratio)

0.9953


### 문제1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후, 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째자리까지 계산)

In [ ]:
# 방법1
from statsmodels.formula.api import logit

# 1) 로지스틱 회귀 모델 생성 및 학습
model = logit("gender ~ age + length + diameter + height + weight", data=train).fit()
print(model.summary())

# 2) 잔차 이탈도 계산
print(round(-2 * model.llf,2))

Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      204
Method:                           MLE   Df Model:                            5
Date:                Sat, 21 Sep 2024   Pseudo R-squ.:                 0.01336
Time:                        05:54:09   Log-Likelihood:                -143.47
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.5662
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.303      -0.605       1.946
age           -0.0482      0.

In [ ]:
# 방법2
from statsmodels.formula.api import glm
import statsmodels.api as sm

# 1) glm 모델 적합 (로지스틱 회귀를 위해 이항 분포 사용)
formula = "gender ~ age + length + diameter + height + weight"
model = glm(formula, data=train, family=sm.families.Binomial()).fit()

# 2) 잔차이탈도 계산
print(model.summary())
print(round(model.deviance,2))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 gender   No. Observations:                  210
Model:                            GLM   Df Residuals:                      204
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -143.47
Date:                Sat, 21 Sep 2024   Deviance:                       286.93
Time:                        05:54:09   Pearson chi2:                     210.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.01833
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.3

### 문제1-3. 독립변수 weight 만 사용해 학습한 로지스틱 회귀모델에서 test데이터의 gender를 예측하고, error rate(오류율)를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import logit

# 데이터셋 분할
train = df.iloc[:210]
test = df.iloc[210:]

# 1) 학습, test데이터를 사용해 예측 (0.5 미만: 0, 0.5 이상 1)
model = logit("gender ~ weight", data=train).fit()
target = test.pop("gender")
pred = model.predict(test) > 0.5

# 2) 실제 값과 예측 값을 사용하여 정확도 계산
acc = accuracy_score(target, pred)

# 3) 오류율 계산
print(round(1-acc,3))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
0.478


In [ ]:
# 다른 풀이 (사이킷런)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

train = df.iloc[:210]
test = df.iloc[210:]

# 1) 학습 데이터와 테스트 데이터 분리
X_train = train[['weight']]
y_train = train['gender']
X_test = test[['weight']]
y_test = test['gender']

# 2) 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 3) 테스트 데이터를 사용해 예측
pred = model.predict(X_test)

# 4) 실제 값과 예측값을 사용하여 정확도 계산
acc = accuracy_score(y_test, pred)

# 5) 오류율 계산
print(round(1 - acc, 3))

0.478


## 문제2

### 문제2-1. ERP와 가장 상관관계 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")
df.head(3)

ERP    Feature1   Feature2   Feature3        CPU
0  135.281047  235.088034  44.462272 -36.532634  20.492369
1  108.003144   36.571914  46.409312 -21.709347  34.169811
2  119.574760   52.239055  66.494894 -30.590820  67.624248

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")

# 1) ERP와 각 변수 사이의 상관계수 계산
df.corr()
# 0.434

ERP  Feature1  Feature2  Feature3       CPU
ERP       1.000000  0.434442  0.076116 -0.059888 -0.201027
Feature1  0.434442  1.000000  0.076148  0.049980 -0.098571
Feature2  0.076116  0.076148  1.000000  0.126589  0.071456
Feature3 -0.059888  0.049980  0.126589  1.000000 -0.035247
CPU      -0.201027 -0.098571  0.071456 -0.035247  1.000000

### 문제2-2. CPU 컬럼이 100미만인 것만 찾아 ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정해 선형회귀 모델을 만들고 적합한 결정계수를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
from statsmodels.formula.api import ols

# 1) CPU가 100 미만인 데이터 필터링
cond = df['CPU'] < 100
df = df[cond]

# 2) 선형회귀 모델 생성: ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정
model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU', data=df).fit()

# 3) 모델 요약 정보 출력
print(model.summary())
print(model.rsquared)
print(model.rsquared_adj)

                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Sat, 21 Sep 2024   Prob (F-statistic):           6.07e-05
Time:                        05:54:10   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.0

### 문제 2-3. 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
# pvalue
# 0.457
print(model.pvalues)

Intercept    0.000003
Feature1     0.000014
Feature2     0.456708
Feature3     0.298206
CPU          0.067968
dtype: float64


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")
sum(df['CPU'] < 100)
df

ERP    Feature1   Feature2   Feature3        CPU
0   135.281047  235.088034  44.462272 -36.532634  20.492369
1   108.003144   36.571914  46.409312 -21.709347  34.169811
2   119.574760   52.239055  66.494894 -30.590820  67.624248
3   144.817864  196.195890  59.828956 -33.400891  87.923476
4   137.351160   75.175619  59.601973 -26.668085  54.367805
..         ...         ...        ...        ...        ...
95  114.131463  105.039508  67.053370 -33.807461  33.747738
96  100.210000  141.114900  51.465875 -25.710380  97.049369
97  135.717410  179.363130  58.744305 -24.294491  13.343943
98  102.538242  217.189747  44.008265 -22.667106   9.680395
99  108.039787  178.875769  55.550838 -25.737240  34.339173

[100 rows x 5 columns]